In [2]:
import pandas as pd

def convert_year_to_4_digits(date_str):
    """Convert 2-digit year to 4-digit year by adding '20' prefix"""
    if isinstance(date_str, str) and '/' in date_str:
        parts = date_str.split('/')
        if len(parts) == 3 and len(parts[2]) == 2:
            parts[2] = '20' + parts[2]
            return '/'.join(parts)
    return date_str

def convert_data_format(df: pd.DataFrame, format: str = '%d/%m/%Y') -> pd.DataFrame:
    df['Compra'] = df['Compra'].apply(lambda x:  convert_year_to_4_digits(x) )
    df['Compra'] = pd.to_datetime(df['Compra'], format=format, errors='coerce')
    df['Inserção'] = df['Inserção'].apply(lambda x:  convert_year_to_4_digits(x) )
    df['Inserção'] = pd.to_datetime(df['Inserção'], format=format, errors='coerce')
    return df
    
def clean_price(price_str):
    """Convert Brazilian price format to numeric value"""
    if isinstance(price_str, str):
        # Remove 'R$' and whitespace, replace comma with dot
        cleaned = price_str.replace('R$', '').strip().replace(',', '.')
        try:
            return float(cleaned)
        except ValueError:
            return None
    return price_str

# Create a function to swap columns 
def swap_columns(df, col1, col2):
    """Swap two columns in a dataframe using values from a temporary dataframe"""
    temp_df = df.copy()

    df[col1] = temp_df[col2]
    df[col2] = temp_df[col1]


year = 2024
# read data
df_2020_original = pd.read_csv(f"../data/sus/2020.csv", sep=';', encoding='ISO-8859-1', quotechar='"')
df_2021_original = pd.read_csv(f"../data/sus/2021.csv", sep=';', encoding='ISO-8859-1', quotechar='"')
df_2022_original = pd.read_csv(f"../data/sus/2022.csv", sep=';', encoding='ISO-8859-1', quotechar='"')
df_2023_original = pd.read_csv(f"../data/sus/2023.csv", sep=';', encoding='utf-8', quotechar='"')
df_2024_original = pd.read_csv(f"../data/sus/2024.csv", sep=';', encoding='utf-8', quotechar='"')

#get copy
df_2020 = df_2020_original.copy()
df_2021 = df_2021_original.copy()
df_2022 = df_2022_original.copy()
df_2023 = df_2023_original.copy()
df_2024 = df_2024_original.copy()

# transform data
df_2020 = convert_data_format(df_2020)
df_2021 = convert_data_format(df_2021)
df_2022 = convert_data_format(df_2022)
df_2023 = convert_data_format(df_2023, format = '%m/%d/%Y')

# transform 2024 data
df_2024['Compra'] = pd.to_datetime('1899-12-30') + pd.to_timedelta(df_2024['Compra'], unit='D')
df_2024['Inserção'] = pd.to_datetime('1899-12-30') + pd.to_timedelta(df_2024['Inserção'], unit='D')


# Define columns for each year
columns_3_first_years = ['ano', 'codigo_br', 'descricao_catmat', 'unidade_de_fornecimento',
    'generico', 'anvisa', 'compra', 'modalidade_da_compra', 'insercao',
    'tipo_compra', 'fabricante', 'cnpj_fabricante', 'fornecedor',
    'cnpj_fornecedor', 'nome_instituicao', 'cnpj_instituicao',
    'municipio_instituicao', 'uf', 'qtd_itens_comprados', 'preco_unitario']


columns_last_2_years = ['nome_instituicao', 'cnpj_instituicao', 'municipio_instituicao', 'uf',
    'compra', 'insercao', 'modalidade_da_compra', 'codigo_br',
    'descricao_catmat', 'unidade_fornecimento', 'generico', 'anvisa',
    'cnpj_fornecedor', 'fornecedor', 'cnpj_fabricante', 'fabricante',
    'qtd_itens_comprados', 'preco_unitario', 'preco_total']


df_2020.columns = columns_3_first_years
df_2021.columns = columns_3_first_years
df_2022.columns = columns_3_first_years
df_2023.columns = columns_last_2_years
df_2024.columns = columns_last_2_years 


# Apply swaps to all three dataframes
for df in [df_2020]:
    # Swap fabricante and cnpj_fabricante
    swap_columns(df , 'fabricante', 'cnpj_fabricante')
    
    # Swap fornecedor and cnpj_fornecedor
    swap_columns(df, 'fornecedor', 'cnpj_fornecedor')
    
    # Swap nome_instituicao and cnpj_instituicao
    swap_columns(df, 'nome_instituicao', 'cnpj_instituicao')


# Apply the conversion to the preco_unitario column
df_2020['preco_unitario'] = df_2020['preco_unitario'].apply(clean_price)
df_2021['preco_unitario'] = df_2021['preco_unitario'].apply(clean_price)
df_2022['preco_unitario'] = df_2022['preco_unitario'].apply(clean_price)
df_2023['preco_unitario'] = df_2023['preco_unitario'].apply(clean_price)
df_2024['preco_unitario'] = df_2024['preco_unitario'].apply(clean_price)

# Convert 'qtd_itens_comprados' to Int64 type

df_2020['qtd_itens_comprados'] = df_2020['qtd_itens_comprados'].astype(str).str.replace('.', '').astype('Int64')
df_2021['qtd_itens_comprados'] = df_2021['qtd_itens_comprados'].astype(str).str.replace('.', '').astype('Int64')
df_2022['qtd_itens_comprados'] = df_2022['qtd_itens_comprados'].astype(str).str.replace('.', '').astype('Int64')
df_2023['qtd_itens_comprados'] = df_2023['qtd_itens_comprados'].astype(str).str.replace('.', '').astype('Int64')
df_2024['qtd_itens_comprados'] = df_2024['qtd_itens_comprados'].astype(str).str.replace('.', '').astype('Int64')

# Standardize column names and order for concatenation
standard_columns = [
    'nome_instituicao', 'cnpj_instituicao', 'municipio_instituicao', 'uf',
    'compra', 'insercao', 'modalidade_da_compra', 'codigo_br',
    'descricao_catmat', 'unidade_fornecimento', 'generico', 'anvisa',
    'cnpj_fornecedor', 'fornecedor', 'cnpj_fabricante', 'fabricante',
    'qtd_itens_comprados', 'preco_unitario', 'preco_total'
]

# Rename 'unidade_de_fornecimento' to 'unidade_fornecimento' in first 3 years to match standard
df_2020 = df_2020.rename(columns={'unidade_de_fornecimento': 'unidade_fornecimento'})
df_2021 = df_2021.rename(columns={'unidade_de_fornecimento': 'unidade_fornecimento'})
df_2022 = df_2022.rename(columns={'unidade_de_fornecimento': 'unidade_fornecimento'})

# Add missing 'preco_total' column to first 3 years (calculate from qty * unit_price)
df_2020['preco_total'] = df_2020['qtd_itens_comprados'] * df_2020['preco_unitario']
df_2021['preco_total'] = df_2021['qtd_itens_comprados'] * df_2021['preco_unitario']
df_2022['preco_total'] = df_2022['qtd_itens_comprados'] * df_2022['preco_unitario']

df_2023['tipo_compra'] = ''
df_2024['tipo_compra'] = ''


# Remove 'ano' column from first 3 years and reorder all dataframes
df_2020 = df_2020.drop(['ano'], axis=1)[standard_columns]
df_2021 = df_2021.drop(['ano'], axis=1)[standard_columns]
df_2022 = df_2022.drop(['ano'], axis=1)[standard_columns]
df_2023 = df_2023[standard_columns]
df_2024 = df_2024[standard_columns]




In [3]:
df = pd.concat([df_2020, df_2021, df_2022, df_2023, df_2024], ignore_index=True)

df['modalidade_da_compra'] = df['modalidade_da_compra'].str.strip()

In [4]:
df['generico'] = df['generico'].apply(lambda x: True if x in ('Sim', 'S') else (False if x in ('Não', 'N') else None))

# Mapeamento das regiões brasileiras
region_mapping = {
    "Norte": ["AC", "AM", "AP", "PA", "RO", "RR", "TO"],
    "Nordeste": ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
    "Centro-Oeste": ["DF", "GO", "MT", "MS"],
    "Sudeste": ["ES", "MG", "RJ", "SP"],
    "Sul": ["PR", "RS", "SC"]
}

df['regiao'] = df['uf'].apply(
    lambda x: next((region for region, states in region_mapping.items() if x in states), None)
)


In [6]:
# Now save to parquet
df.to_csv('../data/sus/sus_data.csv', index=False)

## Municípios

In [7]:
municipios = pd.read_csv('../data/sus/municipios.csv')
estados = pd.read_csv('../data/sus/estados.csv')

# Normalize municipality names to uppercase and remove accents
municipios['nome'] = municipios['nome'].str.upper().str.normalize('NFD').str.encode('ascii', errors='ignore').str.decode('ascii')

municipios = pd.merge(municipios, estados[['codigo_uf', 'uf']], on='codigo_uf', how='left', suffixes=('', '_estado'))

municipios = municipios[['nome', 'latitude', 'longitude', 'uf']]
df['municipio_instituicao'] = df['municipio_instituicao']\
    .str.replace('MUNICIPIO DE ', '', regex=False)\
    .str.replace('FUNDO MUNICIPAL DE ', '', regex=False)\
    .str.rstrip()

df_geo = pd.merge(df, municipios, left_on=['municipio_instituicao','uf'], right_on=['nome', 'uf'], how='left')

df_geo.to_csv('../data/sus/sus_data_geo.csv', index=False)

## API

In [48]:
import requests

url = "https://apidadosabertos.saude.gov.br/economia-da-saude/bps?limit=100&offset=1000"

response =  requests.get(url)

In [49]:
response.json()

{'bps': [{'insercao': '45582',
   'municipio_da_instituicao': 'SAO JOSE DE CAIANA',
   'quantidade_de_itens_comprados': '1000',
   'preco_unitario': '2,84',
   'codigo_br': '268504',
   'preco_total': '2840',
   'unidade_de_fornecimento': 'AMPOLA',
   'fabricante': 'HIPOLABOR FARMACEUTICA LTDA',
   'cnpj_da_instituicao': '10.785.644/0001-96',
   'generico': 'N',
   'fornecedor': 'ALLFAMED COMERCIO ATACADISTA DE MEDICAMENTOS LTDA',
   'cnpj_do_fabricante': '19.570.700/0000-00',
   'modalidade_da_compra': 'Pregão',
   'descricao_catmat': 'ONDANSETRONA CLORIDRATO, DOSAGEM:2 MG/ML, INDICAÇÃO:INJETÁVEL',
   'anvisa': None,
   'compra': '45294',
   'nome_da_instituicao': 'FUNDO MUNICIPAL DE SAUDE DE SAO JOSE DE CAIANA',
   'cnpj_fornecedor': '31.187.918/0001-15',
   'uf': 'PB'},
  {'insercao': '45477',
   'municipio_da_instituicao': 'DOIS VIZINHOS',
   'quantidade_de_itens_comprados': '1000',
   'preco_unitario': '5,01',
   'codigo_br': '342132',
   'preco_total': '5010',
   'unidade_de_forn